In [26]:
import sys
!{sys.executable} -m pip install selenium
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install BeautifulSoup
!{sys.executable} -m pip install webdriver_manager

  Using cached https://files.pythonhosted.org/packages/40/f2/6c9f2f3e696ee6a1fb0e4d7850617e224ed2b0b1e872110abffeca2a09d4/BeautifulSoup-3.2.2.tar.gz
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\jjfri\AppData\Local\Temp\pip-install-17jyw13e\BeautifulSoup\setup.py", line 3
        "You're trying to run a very old release of Beautiful Soup under Python 3. This will not work."<>"Please use Beautiful Soup 4, available through the pip package 'beautifulsoup4'."
                                                                                                        ^
    SyntaxError: invalid syntax
    
    ----------------------------------------


Command "python setup.py egg_info" failed with error code 1 in C:\Users\jjfri\AppData\Local\Temp\pip-install-17jyw13e\BeautifulSoup\


  Using cached https://files.pythonhosted.org/packages/0d/92/6d990e73dbdfcdfcc06f09d52a1ec9a018f851c6c3bd1fe605c6842adf2b/webdriver_manager-3.8.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/30/5f/2e5c564bd86349fe6b82ca840f46acf6f4bb76d79ba9057fce3d3e008864/python_dotenv-0.20.0-py3-none-any.whl
  Found existing installation: urllib3 1.26.10
    Uninstalling urllib3-1.26.10:
      Successfully uninstalled urllib3-1.26.10


selenium 4.3.0 has requirement urllib3[secure,socks]~=1.26, but you'll have urllib3 1.24.3 which is incompatible.


In [35]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

products=[] #List to store name of the product
prices=[] #List to store price of the product
ratings=[] #List to store rating of the product
driver.get("https://www.nytimes.com/2022/07/17/world/europe/europe-uk-heat-wave.html")

content = driver.page_source
soup = BeautifulSoup(content)

c:\Users\jjfri\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


NameError: name 'name' is not defined

In [37]:
for a in soup.findAll('section',href=True, attrs={'name':'articleBody'}):
    name=a.find('div', attrs={'class':'css-s99gbd'})
    products.append(name.text)
    print(a)
    print(name)
    print(products)
# print(a)
print(products)
df = pd.DataFrame({'Product Name':products}) 
print(df.head(20))
#df.to_csv('products.csv', index=False, encoding='utf-8')

[]
Empty DataFrame
Columns: [Product Name]
Index: []


In [13]:
import pandas as pd
import numpy as np

T1 = """TOKYO (Reuters) - Britain and Japan said on Thursday they would cooperate in countering the threat posed by North Korea, two days after it fired a missile over northern Japan, and will call on China to exert its leverage. Prime Minister Theresa May, looking to strengthen relations with one of her closest allies ahead of Brexit, is visiting Japan as it responds to an increasing military threat. Terming North Korea s missile program  a global threat , Prime Minister Shinzo Abe told a news conference that Japan and Britain would cooperate.  It is very meaningful that Prime Minister May and I agreed to further strengthen pressure on North Korea and to call on China to play a larger role,  he added. May agreed, noting that China, North Korea s lone major ally, had been involved in U.N. Security Council debate earlier this week.   China does have a particular position in this, they have leverage on North Korea and I believe we should be encouraging China to exercise that leverage to do what we all want - which is to ensure that North Korea is not conducting these illegal acts.  May toured Japan s flagship Izumo helicopter carrier for a military briefing with Minister of Defence Itsunori Onodera before attending a national security meeting. May and Abe agreed on a joint declaration on security cooperation, including plans for British soldiers to take part in military exercises on Japanese soil and for collaboration to address the threat of cyber and militant attacks when Japan hosts the Olympics in 2020.  North Korea featured heavily in the talks after it launched a ballistic missile on Tuesday that passed over Japanese territory, prompting international condemnation.  May s office had said the two leaders were expected to discuss the possibility of further sanctions on North Korea, but neither Abe nor May touched on the issue at the news conference. The Global Times, a publication of the official People s Daily of China s ruling Communist Party, criticized an earlier comment of May s comment calling for more pressure from China.  Beijing does not need London to teach it how to deal with North Korea,  the newspaper said. Asked about the United States, Japan and Britain looking to impose new sanctions on North Korea, Chinese Foreign Ministry spokeswoman Hua Chunying said the situation could only be resolved peacefully through dialogue.  We think it is regrettable that some countries selectively overlook the relevant Security Council resolutions  demand to advance dialogue, and stubbornly emphasize pressure and sanctions,  she told a daily news briefing.  OUTWARD-LOOKING  Apart from security, May s trip has focused on trade and investment. She is keen to convince nervy investors that Britain s exit from the European Union will not make it a less attractive business partner. Both May and Abe addressed a delegation of British business leaders and senior representatives from major Japanese investors in Britain, such as carmakers Nissan, Toyota and conglomerate Hitachi. Abe told the gathering that May had assured him Britain s negotiations on leaving the European Union would be transparent. May said Japanese investment after Britain s vote to leave the EU was a vote of confidence and she pledged to build close trade ties with Japan.  I very much welcome the commitment from Japanese companies such as Nissan, Toyota, Softbank and Hitachi,  May said.  I am determined that we will seize the opportunity to become an ever more outward-looking global Britain, deepening our trade relations with old friends and new allies.   During a two-hour train ride between Kyoto and Tokyo late on Wednesday, the two leaders discussed Brexit, with May talking Abe through the details of a series of papers published in recent weeks setting out her negotiating position. May said on Wednesday Japan s upcoming trade deal with the EU could offer a template for a future Japan-Britain trade agreement, the latest attempt to show investors that Brexit will not lead to an overnight change in business conditions. Japan has been unusually open about its concerns over Brexit, worrying that 40 billion pounds ($51.68 billion) of Japanese investment in the British economy could suffer if trading conditions change abruptly when Britain leaves the bloc."""

T2 = '''NEW DELHI (Reuters) - Indian Prime Minister Narendra Modi plans to reshuffle his cabinet over the next two days and some mid-level ministers have offered to quit to pave the way for changes, sources in the party and aides to the ministers said. A revamp of the cabinet has been on the cards for months because some ministers are holding multiple portfolios which is seen as a drag on efficiency. Modi is also seeking to improve governance before he seeks re-election in 2019.     With economic growth slowing to its slowest pace in three years, Modi is coming under pressure to deliver the promises he made in 2014.     There was no official announcement of the cabinet revamp but  sources Reuters spoke to said if it took place it would have to be done before Modi leaves on a foreign tour on Sunday.     The death of Environment Minister Anil Madhav Dave and the election of Minister for Urban Development M. Venkaiah Naidu as vice-president opened vacancies, giving Modi an opportunity to bring in members from regional political parties.     According to four party sources, Sanjeev Balyan, minister of state for water resources, Rajiv Pratap Rudy, minister of state for skill development and entrepreneurship, Kalraj Mishra, minister of micro, small and medium enterprises, and Mahendra Nath Pandey minister of state for human resource development have all offered to resign.'''


T3 = "hello world"
dict = {'label':['unknown', 'unknown', 'unknown'],
        'text':[T1, T2, T3],
       }
df = pd.DataFrame(dict)
df.head()
df[df.text.apply(lambda x: len(x) > 50)]


,label,text
0,unknown,TOKYO (Reuters) - Britain and Japan said on Th...
1,unknown,NEW DELHI (Reuters) - Indian Prime Minister Na...
